In [2]:
import numpy as np
import datetime
import cv2 
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Activation,Conv2D,Dense,MaxPool2D,Dropout,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
from IPython.display import Image, display

from tensorflow.keras.applications import InceptionV3,Xception,VGG16
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Model
from keras.layers import Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

Base_directory = '/kaggle/input/flamedatasetfireclassification'
test_path = 'Test/Test'
Training_path = 'Training/Training'
input_shape = (254,254,3)
batch = 16
labels = ['Fire','No_Fire']
Full_Training_path = '{0}/{1}'.format(Base_directory,Training_path)
Full_Test_path = '{0}/{1}'.format(Base_directory,test_path)
train_images = ImageDataGenerator(rotation_range=45,
                                 horizontal_flip=True,
                                 vertical_flip=True,
                                 rescale=1.0/255,
                                 zoom_range=0.4,
                                 shear_range=0.2,
                                 fill_mode='nearest',
                                 validation_split=0.2)

train_generator = train_images.flow_from_directory(Full_Training_path, 
                                               target_size=(254,254),
                                               color_mode='rgb',
                                               class_mode='binary',
                                               batch_size=batch,
                                               #shuffle=True,
                                               subset='training')

validation_generator = train_images.flow_from_directory(Full_Training_path, 
                                                    target_size=(254,254),
                                                    color_mode='rgb',
                                                    class_mode='binary',
                                                    batch_size=batch,
                                                   # shuffle=True,
                                                    subset='validation')

test_images = ImageDataGenerator(rescale=1.0/255)

test_generator = test_images.flow_from_directory(Full_Test_path, 
                                                target_size=(254,254), 
                                                color_mode='rgb', 
                                                class_mode='binary',
                                                shuffle=False,
                                                batch_size=batch)


# create model
img_input = Input(shape=input_shape)

model = VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=img_input,
    input_shape=input_shape,
    pooling='avg'
)
last_layer = model.output
x = Flatten(name='flatten')(last_layer)
x = Dense(1024, activation='relu', name='fc1')(x)
out = Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
             ='linear')(x)  ## 2 classes
model = Model(img_input, out)

# for layer in model.layers[:-80]:
     layer.trainable = False
    
from keras.callbacks import Callback

class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        testset = self.test_data
        loss, acc = self.model.evaluate(testset, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))
        
my_callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1),
    ModelCheckpoint(filepath='VGG16_{epoch:02d}_{val_accuracy:.04f}.h5',
                    save_best_only=False),
    TestCallback(test_generator)]       

model.compile( loss = 'hinge',
      optimizer='AdaMax',  #AdaMax
      metrics='accuracy')

model_json = model.to_json()
with open("VGG16_saved_model.json", "w") as json_file:
    json_file.write(model_json)
    
    
history = model.fit(train_generator,
            epochs=50,
            validation_data = validation_generator,            
            callbacks=my_callbacks,
            verbose=1    )

Found 31501 images belonging to 2 classes.
Found 7874 images belonging to 2 classes.
Found 8617 images belonging to 2 classes.
Epoch 1/50
1969/1969 [==============================] - 766s 386ms/step - loss: 0.1010 - accuracy: 0.9655 - val_loss: 0.0633 - val_accuracy: 0.9765

Testing loss: 0.4057602882385254, acc: 0.8082859516143799

Epoch 2/50
1969/1969 [==============================] - 673s 342ms/step - loss: 0.0230 - accuracy: 0.9914 - val_loss: 0.0353 - val_accuracy: 0.9868

Testing loss: 0.3742777109146118, acc: 0.8116514086723328

Epoch 3/50
1969/1969 [==============================] - 644s 327ms/step - loss: 0.0185 - accuracy: 0.9920 - val_loss: 0.0596 - val_accuracy: 0.9793

Testing loss: 0.4517914652824402, acc: 0.8014389872550964

Epoch 4/50
1969/1969 [==============================] - 659s 335ms/step - loss: 0.0179 - accuracy: 0.9924 - val_loss: 0.0976 - val_accuracy: 0.9577

Testing loss: 0.3798963725566864, acc: 0.8354415893554688

Epoch 5/50
1969/1969 [===================